In [183]:
import math
import random

In [184]:
def exponencial(lde,t):
    return -((1/lde)*math.log(random.random()))

In [185]:
def poisson(ldpp,t):
    return t-((1/ldpp)*math.log(random.random()))

In [186]:
class task:
    def __init__(self, ta):
        self.t_llegada = ta
        self.t_atendido = float('Inf')
        self.t_salida = float('Inf')
    
    #devuelve el tiempo total del task
    def get_tiempoT(self):
        return (self.t_salida - self.t_llegada)
    
    #devuelve el tiempo en servidor
    def get_tiempoEnServ(self):
        return (self.t_salida - self.t_atendido)
    
    #devuelve tiempo de salida

In [187]:
class server:
    def __init__(self):
        self.bussy = False
        self.task_running = False
        self.tasks = []
        self.Tp = 0
    
    def assign_task(self, taske):
        self.bussy = True
        self.task_running = taske
        self.tasks.append(taske)
        
        

In [188]:
class SystemSim:
    def __init__(self, T, ldp, lde, servidores):
        #flags
        self.serv_working = True #cambia hasta el caso 4
        #parametros cambian
        self.T = T
        self.ldp = ldp
        self.lde = lde
        
        #parametros constantes
        self.t = 0
        self.Na = 0
        self.Nd = 0
        self.n = 0
        self.ta = poisson(ldp,self.t)
        
        #listas
        self.cola_eventos = []
        self.tiempos_salida = []
        self.servidores = []
        
        #inicializo objetos servidor
        for i in range(servidores):
            self.servidores.append(server())
            self.tiempos_salida.append(float('Inf'))
            
    #Verifica si aun hay tiempo de ejecucion
    def get_tiempoDisp(self):
        if(self.t < self.T and self.serv_working):
            return True
        else:
            return False
    
    #agrega un nuevo objeto a la cola
    def add_task(self,ta):
        servidor_pos = self.get_free()
        #existe un servidor desocupado
        self.cola_eventos.append(task(ta)) #### se mete task a la cola
        if(servidor_pos > -1):
            # si hay servidor libre se jala primer task de la cola
            self.servidores[servidor_pos].assign_task(self.cola_eventos[0])
             # se le asigna tiempo de salida
            self.cola_eventos[0].t_salida = self.t + exponencial(self.lde, self.t)
            # si se jaló a algún servidor se elimina de la lista de cola
            self.cola_eventos.pop(0)
            self.servidores[servidor_pos].task_running.t_atendido = ta #se atiende solo llegando
    
    #identifica que servidor esta libre
    def get_free(self):
        #devuelve -1 si no hay libres
        for i in range(len(self.servidores)):
            if(not self.servidores[i].bussy):
                return i
        return -1
    
    #devuelve el tiempo de salida mas pequeño
    def get_td(self):
        td_min = float('Inf')
        td_servidor = -1
        for i in range(len(self.servidores)):
            if((self.servidores[i].task_running != False) and (self.servidores[i].task_running.t_salida < td_min)):
                td_min = self.servidores[i].task_running.t_salida
                td_servidor = i
        return (td_min,td_servidor)
        
    #debug block
    def deb(self):
        print("t: ", self.t, "\tta: ", self.ta, "\ttb: ", self.get_td()[0])
        

In [189]:
def caso1(world):
    print("CASO 1")
    if(world.ta <= world.T):
        world.deb()
        world.t = world.ta
        world.Na += 1
        world.n += 1
        world.ta = poisson(world.ldp,world.t)
        world.add_task(world.t)
    

In [190]:
def caso2(world):
    print("CASO 2")
    world.deb()
    world.t = world.get_td()[0]
    servidor = world.get_td()[1]
    world.n -= 1
    world.Nd += 1
    world.servidores[servidor].bussy = False
    if(not world.get_tiempoDisp()):
        world.servidores[servidor].Tp = max(world.t - world.T, 0)

In [191]:
def caso4(world):
    print("CASO 4")
    world.serve_working = False

In [192]:
def selector(world):
    if(world.ta < world.get_td()[0]):
        # El siguiente evento es una llegada de una solicitud al 
        # sistema y aún no es la hora de cierre. 
        caso1(world)
    elif(world.get_td()[0] < world.ta):
        #print(world.get_td()[0])
        #print(world.ta)
        #print(world.t)
        
        # El siguiente evento es una salida de una solicitud del
        # sistema
        caso2(world)
    else:
        caso4(world)

In [193]:
def simulador(T,ldp,lde,servidores):
    #se crea el sistema con todos los parametros
    world = SystemSim(T,ldp,lde,servidores)
    
    while(world.get_tiempoDisp()):
        selector(world)
        

In [194]:
simulacion = simulador(1,2400/60,100,2)

CASO 1
t:  0 	ta:  0.042442789006507754 	tb:  inf
CASO 2
t:  0.042442789006507754 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.05

t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239

CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0

t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239

t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239

t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239

t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239

t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239341031409826
CASO 2
t:  0.058239341031409826 	ta:  0.10342026456766923 	tb:  0.058239

KeyboardInterrupt: 